# Sleep Duration Prediction Model
This notebook predicts sleep duration based on daily activity metrics.

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

In [14]:
data = pd.DataFrame({
    'customer_id': ['CUST1', 'CUST1', 'CUST1', 'CUST1', 'CUST1', 'CUST1', 
                   'CUST2', 'CUST2', 'CUST2', 'CUST2', 'CUST2', 'CUST2',
                   'CUST3', 'CUST3', 'CUST3'],
    'date': ['24/02/25', '23/02/25', '22/02/25', '21/02/25', '20/02/25', '19/02/25',
            '24/02/25', '23/02/25', '22/02/25', '21/02/25', '20/02/25', '19/02/25',
            '12/02/25', '11/02/25', '10/02/25'],
    'sleep_duration': [570, 421, 387, 593, 321, 358,
                      350, 434, 541, 391, 563, 349,
                      353, 517, 569],
    'steps': [10191, 5466, 13322, 7433, 11396, 7558,
             6899, 10393, 12513, 10486, 13226, 8943,
             8843, 10675, 12629],
    'resting_heart_rate': [78, 72, 71, 77, 61, 59,
                          72, 53, 70, 77, 63, 53,
                          71, 51, 72]
})


data.head()

,customer_id,date,sleep_duration,steps,resting_heart_rate
0,CUST1,24/02/25,570,10191,78
1,CUST1,23/02/25,421,5466,72
2,CUST1,22/02/25,387,13322,71
3,CUST1,21/02/25,593,7433,77
4,CUST1,20/02/25,321,11396,61


In [15]:
def create_customer_history(data):
    """Create features from customer history"""
    history = data.copy()
    
    # Convert date to datetime
    history['date'] = pd.to_datetime(history['date'], format='%d/%m/%y')
    
    # Extract day of week features
    history['day_of_week'] = history['date'].dt.dayofweek
    history['is_weekend'] = history['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    # Previous day metrics
    history['prev_sleep'] = history.groupby('customer_id')['sleep_duration'].shift(1)
    history['prev_steps'] = history.groupby('customer_id')['steps'].shift(1)
    history['prev_heart_rate'] = history.groupby('customer_id')['resting_heart_rate'].shift(1)
    
    # Add rolling statistics and other features
    for col in ['sleep_duration', 'steps', 'resting_heart_rate']:
        col_prefix = col.split('_')[0]
        history[f'{col_prefix}_roll_mean_3d'] = history.groupby('customer_id')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
        )
        history[f'{col_prefix}_roll_std_3d'] = history.groupby('customer_id')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).std().shift(1).fillna(0)
        )
        history[f'{col_prefix}_change'] = history.groupby('customer_id')[col].diff().fillna(0)
    
    # Interaction features
    history['steps_per_hr'] = history['steps'] / history['resting_heart_rate'].replace(0, 1)
    
    return history

In [16]:
def train_sleep_prediction_model():
    """Train the sleep prediction model"""
    
    customer_history = create_customer_history(data)
    
    
    features = [
        'prev_sleep', 'prev_steps', 'prev_heart_rate',
        'sleep_roll_mean_3d', 'steps_roll_mean_3d',
        'sleep_roll_std_3d', 'steps_roll_std_3d',
        'sleep_change', 'steps_change',
        'steps_per_hr', 'is_weekend', 'day_of_week'
    ]
    
    
    X = customer_history[features].dropna()
    y = customer_history.loc[X.index, 'sleep_duration']
    
    
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('ridge', Ridge(alpha=1.0))
    ])
    
    model.fit(X, y)
    return model, features

In [17]:
model, features = train_sleep_prediction_model()
print("Model training completed!")

Model training completed!


## Interactive Prediction

In [18]:
from IPython.display import display
from ipywidgets import widgets

customer_id = widgets.Text(value='CUST1', description='Customer ID:')
steps = widgets.IntText(value=5000, description='Steps:')
heart_rate = widgets.IntText(value=70, description='Heart Rate:')
sleep_duration = widgets.IntText(value=420, description='Sleep (min):')

def on_predict_click(b):
    prediction_data = {
        'customer_id': customer_id.value,
        'date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'sleep_duration': sleep_duration.value,
        'steps': steps.value,
        'resting_heart_rate': heart_rate.value
    }
    prediction = predict_next_day_sleep(prediction_data)
    print(f"Predicted sleep duration for tomorrow: {prediction} minutes")

predict_button = widgets.Button(description='Predict')
predict_button.on_click(on_predict_click)
#hi
display(customer_id, steps, heart_rate, sleep_duration, predict_button)

Text(value='CUST1', description='Customer ID:')

IntText(value=5000, description='Steps:')

IntText(value=70, description='Heart Rate:')

IntText(value=420, description='Sleep (min):')

Button(description='Predict', style=ButtonStyle())